# Problem

Popular question and answer (qna) site -  stackoverflow (+ their sister sites) allows for download of monthly data dumps from https://archive.org/details/stackexchange.

With this data, can we classify the questions/answers based on

* Conceptual v/s howto question 
* Beginner v/s intermediate v/s hard/trick
* A particular question is associated with another question in terms of the next things to do or perhaps the pre-requisites?
* Predict the next question a user may ask based on this current search

The taxanomy could be a useful layout of the land for a student of the area.

# Schema

The schema for their data is located @ https://ia800500.us.archive.org/22/items/stackexchange/readme.txt.
    
Unfortunately, the data is dumped in an XML format and there is preliminary effort to convert that data into CSV format. We have written a converter (convert2csv.py) for the tables of interest.

The schemas for the tables of interest are shown below.


## Posts
-----------
- Id
- PostTypeId
  - 1: Question
  - 2: Answer
- ParentID (only present if PostTypeId is 2)
- AcceptedAnswerId (only present if PostTypeId is 1)
- CreationDate
- Score
- ViewCount
- Body
- OwnerUserId
- LastEditorUserId
- LastEditorDisplayName="Jeff Atwood"
- LastEditDate="2009-03-05T22:28:34.823"
- LastActivityDate="2009-03-11T12:51:01.480"
- CommunityOwnedDate="2009-03-11T12:51:01.480"
- ClosedDate="2009-03-11T12:51:01.480"
- Title=
- Tags=
- AnswerCount
- CommentCount
- FavoriteCount

## Comments
---------------------------
- Id
- PostId
- Score
- Text, e.g.: "@Stu Thompson: Seems possible to me - why not try it?"
- CreationDate, e.g.:"2008-09-06T08:07:10.730"
- UserId

## Post History
---------------------------
- Id
- PostHistoryTypeId
    - 1: Initial Title - The first title a question is asked with.
    - 2: Initial Body - The first raw body text a post is submitted with.
    - 3: Initial Tags - The first tags a question is asked with.
    - 4: Edit Title - A question's title has been changed.
    - 5: Edit Body - A post's body has been changed, the raw text is stored here as markdown.
    - 6: Edit Tags - A question's tags have been changed.
    - 7: Rollback Title - A question's title has reverted to a previous version.
    - 8: Rollback Body - A post's body has reverted to a previous version - the raw text is stored here.
    - 9: Rollback Tags - A question's tags have reverted to a previous version.
    - 10: Post Closed - A post was voted to be closed.
    - 11: Post Reopened - A post was voted to be reopened.
    - 12: Post Deleted - A post was voted to be removed.
    - 13: Post Undeleted - A post was voted to be restored.
    - 14: Post Locked - A post was locked by a moderator.
    - 15: Post Unlocked - A post was unlocked by a moderator.
    - 16: Community Owned - A post has become community owned.
    - 17: Post Migrated - A post was migrated.
    - 18: Question Merged - A question has had another, deleted question merged into itself.
    - 19: Question Protected - A question was protected by a moderator
    - 20: Question Unprotected - A question was unprotected by a moderator
    - 21: Post Disassociated - An admin removes the OwnerUserId from a post.
    - 22: Question Unmerged - A previously merged question has had its answers and votes restored.
- PostId
- RevisionGUID: At times more than one type of history record can be recorded by a single action.  
- CreationDate: "2009-03-05T22:28:34.823"
- UserId
- UserDisplayName: populated if a user has been removed and no longer referenced by user Id
- Comment: This field will contain the comment made by the user who edited a post
- Text: A raw version of the new value for a given revision. 
- CloseReasonId
    - 1: Exact Duplicate - This question covers exactly the same ground as earlier questions on this topic; its answers may be merged with another identical question.
    - 2: off-topic
    - 3: subjective
    - 4: not a real question
    - 7: too localized
       
       
## Users
---------------------------
 - Id
 - Reputation
 - CreationDate
 - DisplayName
 - EmailHash
 - LastAccessDate
 - WebsiteUrl
 - Location
 - Age
 - AboutMe
 - Views
 - UpVotes
 - DownVotes
       

# Conversion from XML to CSV

Run python convert2csv.py to convert each of the xml files to their CSV equivalents. For columns/attributes which contain textual data, the converter encodes them with base64 encoding so that handling of quotes and special characters (separators) is avoided. 

When the data is read back into the dataframe, the corresponding decode (from base64) needs to happen. The converter also creates a sample file of 100 rows for each xml data dump converted.

In [73]:
#imports
import pandas as pd
import base64
import math
import re
import gensim
from gensim import corpora, models
from gensim.models import word2vec
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup 
from nltk.stem.porter import PorterStemmer

%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()


SAMPLE_SIZE = 500

In [61]:
posts = pd.read_csv('posts.csv',nrows=SAMPLE_SIZE).dropna(subset=['Body','Title'])
posts['Body'] = posts['Body'].apply(lambda x : BeautifulSoup(base64.b64decode(x),"lxml").get_text())
posts['Title'] = posts['Title'].apply(lambda x : BeautifulSoup(base64.b64decode(x),"lxml").get_text())
posts[['Body','Title', 'Tags']].head(5)

,Body,Title,Tags
0,When should I use can? When should I use could...,"When do I use ""can"" or ""could""?",<word-choice><tenses><politeness><subjunctive-...
1,"Doesn't ""quint"" mean ""five""? What does that h...","Where does the ""quint"" in ""quintessential"" com...",<etymology>
2,"Which is the correct use of these two words, a...","When should I use ""shall"" versus ""will""?",<future-tense><shall-future><will-future>
4,"I think most folk happily use either ""while"" o...","When did ""while"" and ""whilst"" become interchan...",<word-choice><grammar><etymology><conjunctions>
5,\n\nI may not be coming in tomorrow... \nI mig...,"""May"" & ""Might"": What's the right context?",<word-choice><grammar><modal-verbs><auxiliary-...


### Notice that we have stripped out the html formatting tags with BeautifulSoup before reassigning back to the dataframe

In [62]:
comments = pd.read_csv('comments.csv',nrows=SAMPLE_SIZE).dropna()
comments['Text'] = comments['Text'].apply(lambda x : BeautifulSoup(base64.b64decode(x),"lxml").get_text())
comments[['Score','Text']].head(5)

,Score,Text
0,9,I think you need to edit the title of your que...
1,12,It's correct when you're accessing a method of...
2,2,"Yes, I would think in almost any context where..."
3,0,"Would you say `It can certainly be ""acceptable..."
4,4,@serg555: Would you expect anything less on a ...


In [63]:
#posthistory = pd.read_csv('posthistory.csv',nrows=SAMPLE_SIZE).dropna(subset=['Text'])
#posthistory['Text'] = posthistory['Text'].apply(lambda x : BeautifulSoup(base64.b64decode(x),"lxml").get_text())
#posthistory[['Text']].head(5)

In [64]:
users = pd.read_csv('users.csv',nrows=SAMPLE_SIZE).dropna(subset=['AboutMe','Location'])
users['AboutMe'] = users['AboutMe'].apply(lambda x : BeautifulSoup(base64.b64decode(x),"lxml").get_text())
users['Location'] = users['Location'].apply(lambda x : BeautifulSoup(base64.b64decode(x),"lxml").get_text())
users[['Location','AboutMe']].head(5)

,Location,AboutMe
0,on the server farm,"Hi, I'm not really a person.\nI'm a background..."
1,"Corvallis, OR",Developer on the Stack Overflow team. Find me...
2,"New York, NY",Developer on the Stack Overflow team.\nWas dub...
3,"Raleigh, NC",I design stuff for Stack Exchange. Also a prof...
4,California,"I slip my front end into the back end, and the..."


## Further cleansing

* Remove (html) tags & carriage returns from the Text field
* Remove stop words (pick up the nltk stop words)
* Use PorterStemmer to stem words

In [65]:
#global
p_stemmer = PorterStemmer()
stop_words = stopwords.words('english')
stop_words.append('use')
#print(stop_words)

In [82]:
class SentenceTokens():
    def __init__(self,df,field):
        self.field = field
        self.df = df
    
    def __iter__(self):
      for index, row in self.df.iterrows():
         raw_sentence = row[self.field].replace('\n','').lower()
         raw_tokens = filter(None, re.split("[ ?.,\"\')()_]+",raw_sentence))
         #stem_tokens = [p_stemmer.stem(tok) for tok in raw_tokens]
         yield [tok for tok in raw_tokens if not tok in stop_words]

#all posts is a list of (list of tokens). The inner list of tokens is created once for each post
allposts = SentenceTokens(posts,'Title')

for p in allposts:
    print(p)

[u'could']
[u'quint', u'quintessential', u'come']
[u'shall', u'versus']
[u'whilst', u'become', u'interchangeable']
[u'may', u'&', u'might', u':', u'right', u'context']
[u'appropriate', u'short', u'form', u'means', u'possession']
[u'words', u'title', u'capitalized']
[u'appropriate', u'end', u'sentence', u'preposition']
[u'term', u'ok/okay', u'come']
[u'proper', u'plural', u'word', u'freshman']
[u'singular', u'innings', u'come']
[u'betwixt', u'trebble', u'etc', u'acceptable', u'american', u'english']
[u'ever', u'acceptable', u'period', u'come', u'quote', u'end', u'sentence']
[u'wrong', u'in/with', u'question']
[u'correct', u'gender-neutral', u'singular', u'pronoun', u'versus', u'versus']
[u'word', u'contracted', u'twice', u'e', u'g', u'ven']
[u'what\u2019s', u'rule', u'using', u'\u201cwho\u201d', u'\u201cwhom\u201d', u'correctly']
[u'ask', u'question', u'politely']
[u'difference', u'gerund', u'participle']
[u'\u201cthat\u201d', u'\u201cwhich\u201d']
[u'acceptable', u'start', u'sentence',

In [67]:
#How frequently each term occurs within each document? We construct a document-term matrix.
dictionary = corpora.Dictionary(allposts)
#print(dictionary.token2id) #maps ids to tokens

In [68]:
#bag of words
corpus = [dictionary.doc2bow(text) for text in allposts]
#corpus, is a list of vectors equal to the number of documents. 
#In each document vector is a series of tuples. 
print(corpus[0])
#(term ID, term frequency) pairs

[(0, 1)]


In [69]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=30)

#num_topics: required. An LDA model requires the user to determine how many topics should be generated. 
#id2word: required. The LdaModel class requires our previous dictionary to map ids to strings.
#passes: optional. The number of laps the model will take through corpus.

In [70]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

In [71]:
@interact(num_topics=5, num_words=6)
def understand(num_topics, num_words):
    return ldamodel.print_topics(num_topics, num_words)
    
#Each generated topic is separated by a comma.
#Within each topic are the three most probable words to appear in that topic.

#interact(understand, x=widgets.IntSlider(min=0,max=300,step=1,value=10));

[(8,
  u'0.051*vs + 0.031*said + 0.031*sentence + 0.031*e + 0.021*great + 0.021*car'),
 (1,
  u'0.038*appropriate + 0.020*difference + 0.020*come + 0.020*means + 0.020*email + 0.020*work'),
 (4,
  u'0.026*employee + 0.026*usage + 0.026*acceptable + 0.014*sentence + 0.014*instead + 0.014*used'),
 (3,
  u'0.020*vs + 0.020*english + 0.020*word + 0.020*line + 0.020*acceptable + 0.020*mark'),
 (5,
  u'0.034*instead + 0.023*gerund + 0.023*singular + 0.023*accident + 0.023*infinitive + 0.023*try')]

In [72]:
pyLDAvis.gensim.prepare(ldamodel,corpus,dictionary)


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      15.506285        1       1 -0.047753 -0.175805
3      12.275762        1       2  0.083529 -0.056523
7      11.421671        1       3  0.064082 -0.026498
8      11.166481        1       4  0.091167  0.054378
6      10.614318        1       5 -0.039684  0.077602
5       9.633095        1       6 -0.137389  0.050529
9       9.433242        1       7  0.055660  0.051852
4       8.131712        1       8  0.008330  0.019355
2       7.402313        1       9 -0.071802 -0.001104
1       4.415121        1      10 -0.006139  0.006215, topic_info=     Category      Freq         Term     Total  loglift  logprob
term                                                            
122   Default  5.000000      correct  5.000000  30.0000  30.0000
328   Default  2.000000  appropriate  2.000000  29.0000  29.0000
68    Default  4.000000      instead  4.000000  28.0000  28.0000
263   Default  7.000000           vs  7.000000  27.0000  27.0000
47    Default  3.000000   acceptable  3.000000  26.0000  26.0000
287   Default  5.000000     sentence  5.000000  25.0000  25.0000
216   Default  2.000000         said  2.000000  24.0000  24.0000
51    Default  4.000000         word  4.000000  23.0000  23.0000
366   Default  3.000000            e  3.000000  22.0000  22.0000
95    Default  2.000000       versus  2.000000  21.0000  21.0000
193   Default  2.000000     employee  2.000000  20.0000  20.0000
295   Default  2.000000        usage  2.000000  19.0000  19.0000
15    Default  5.000000   difference  5.000000  18.0000  18.0000
71    Default  2.000000     singular  2.000000  17.0000  17.0000
195   Default  2.000000        means  2.000000  16.0000  16.0000
355   Default  1.000000     compared  1.000000  15.0000  15.0000
266   Default  1.000000    different  1.000000  14.0000  14.0000
61    Default  2.000000       phrase  2.000000  13.0000  13.0000
83    Default  3.000000        words  3.000000  12.0000  12.0000
228   Default  3.000000         come  3.000000  11.0000  11.0000
191   Default  2.000000       gerund  2.000000  10.0000  10.0000
331   Default  2.000000     question  2.000000   9.0000   9.0000
54    Default  1.000000         work  1.000000   8.0000   8.0000
334   Default  2.000000          way  2.000000   7.0000   7.0000
99    Default  3.000000         name  3.000000   6.0000   6.0000
221   Default  1.000000        email  1.000000   5.0000   5.0000
230   Default  1.000000     accident  1.000000   4.0000   4.0000
146   Default  1.000000   infinitive  1.000000   3.0000   3.0000
19    Default  1.000000          try  1.000000   2.0000   2.0000
74    Default  1.000000         less  1.000000   1.0000   1.0000
...       ...       ...          ...       ...      ...      ...
221   Topic10  0.483912        email  1.690569   1.8692  -3.9175
195   Topic10  0.483912        means  2.195494   1.6079  -3.9175
228   Topic10  0.483922         come  3.483973   1.1461  -3.9174
15    Topic10  0.483941   difference  5.186191   0.7483  -3.9174
364   Topic10  0.043991     football  1.119278  -0.1163  -6.3154
321   Topic10  0.043991      “which”  1.119278  -0.1163  -6.3154
93    Topic10  0.043991       soccer  1.119278  -0.1163  -6.3154
97    Topic10  0.043991       “that”  1.119278  -0.1163  -6.3154
278   Topic10  0.043991       colour  1.119278  -0.1163  -6.3154
241   Topic10  0.043991        color  1.119278  -0.1163  -6.3154
77    Topic10  0.043991     spelling  1.119278  -0.1163  -6.3154
327   Topic10  0.043991     freshman  1.119278  -0.1163  -6.3154
305   Topic10  0.043991       buyer*  1.119278  -0.1163  -6.3154
118   Topic10  0.043991      remorse  1.119278  -0.1163  -6.3154
360   Topic10  0.043991      details  1.119278  -0.1163  -6.3154
26    Topic10  0.043991       —which  1.119278  -0.1163  -6.3154
152   Topic10  0.043991       bother  1.119278  -0.1163  -6.3154
68    Topic10  0.044016      instead  4.061382  -1.4046  -6

In [84]:
model = word2vec.Word2Vec(allposts, size=200)
model.most_similar(positive = ['correct'])


[(u'difference', 0.14371222257614136),
 (u'english', 0.10942308604717255),
 (u'words', 0.0999104455113411),
 (u'sentence', -0.0063072070479393005),
 (u'vs', -0.015772536396980286),
 (u'instead', -0.0242946557700634),
 (u'word', -0.0395488366484642),
 (u'come', -0.040346503257751465),
 (u'e', -0.07119997590780258),
 (u'used', -0.09382086992263794)]

## References

* https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html
* LDA Viz - http://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf
* This dashboard @ https://github.com/dhruvaray/soml
    